In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.init as init

import numpy as np
import matplotlib.pyplot as plt
import copy
from ast import Param
import json
import math

from utils.prune import apply_mask
from utils.count_improvement import improvements
from utils.normalize import normalize_weights

In [2]:
# Verificar si la GPU está disponible y establecer el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [4]:
net = Net().to(device)
varianzas_net = Net().to(device)
individuo = Net().to(device)

In [5]:
# guardar individuo params in txt
def save_params(individuo, filename = "individuo.txt"):
    with open(filename, "w") as f:
        #I want to see all params
        for name, param in individuo.named_parameters():
            f.write(f"{name} {param}\n")
            

In [6]:
def make_to_binary(individuo):
    total_elements = 0
    total_ones = 0

    for name, param in individuo.named_parameters():
        if "weight" in name or "bias" in name:
            # Calcula el percentil 30 directamente sobre los valores de los parámetros
            threshold = param.quantile(0.7)

            # Binariza: 1 para valores mayores al percentil 30, 0 para los demás
            binary_param = torch.where(param > threshold, 
                                       torch.tensor(1, dtype=torch.float).to(param.device), 
                                       torch.tensor(0, dtype=torch.float).to(param.device))
            
            # Actualiza los valores binarizados en los parámetros
            param.data = binary_param
            
            # Actualiza contadores
            total_elements += binary_param.numel()
            total_ones += (binary_param == 1).sum().item()

    # Calcula los porcentajes
    percentage_ones = (total_ones / total_elements) * 100
    percentage_zeros = 100 - percentage_ones

    # Muestra el resultado
    print(f"Porcentaje de 1s: {percentage_ones:.2f}%")
    print(f"Porcentaje de 0s: {percentage_zeros:.2f}%")
    
    return individuo


In [7]:
def mutate_weights(individuo, mutation_rate=0.05):
    for name, param in individuo.named_parameters():
        if "weight" in name or "bias" in name:
            # Copia los parámetros binarios
            binary_param = param.data.clone()

            # Encuentra índices de 1s y 0s
            ones_indices = (binary_param == 1).nonzero(as_tuple=True)[0]
            zeros_indices = (binary_param == 0).nonzero(as_tuple=True)[0]

            # Determina cuántos elementos mutar
            num_ones_to_flip = int(len(ones_indices) * mutation_rate)
            num_zeros_to_flip = int(len(zeros_indices) * mutation_rate)

            # Selecciona índices aleatorios para mutar
            if num_ones_to_flip > 0:
                selected_ones = ones_indices[torch.randperm(len(ones_indices))[:num_ones_to_flip]]
                binary_param[selected_ones] = 0

            if num_zeros_to_flip > 0:
                selected_zeros = zeros_indices[torch.randperm(len(zeros_indices))[:num_zeros_to_flip]]
                binary_param[selected_zeros] = 1

            # Actualiza los valores binarizados
            param.data = binary_param

    # Calcula el porcentaje final de 1s y 0s
    total_elements = 0
    total_ones = 0

    for name, param in individuo.named_parameters():
        if "weight" in name or "bias" in name:
            binary_param = param.data
            total_elements += binary_param.numel()
            total_ones += (binary_param == 1).sum().item()

    percentage_ones = (total_ones / total_elements) * 100
    percentage_zeros = 100 - percentage_ones

    print(f"Porcentaje de 1s después de mutación: {percentage_ones:.2f}%")
    print(f"Porcentaje de 0s después de mutación: {percentage_zeros:.2f}%")

    return individuo



def compare_models(model1, model2):
    identical = True
    for (name1, param1), (name2, param2) in zip(model1.named_parameters(), model2.named_parameters()):
        print(f"Comparando {name1} y {name2}")
        print(f"¿Los tensores son iguales? {torch.equal(param1.data, param2.data)}")
        if not torch.equal(param1.data, param2.data):
            identical = False
            break

    if identical:
        print("Las redes son idénticas.")
    else:
        print("Las redes son diferentes.")

In [8]:
save_params(individuo)
individuo_binario = make_to_binary(individuo)
save_params(individuo, "individuo_binario.txt")

model_perturbed = mutate_weights(individuo_binario)

save_params(model_perturbed, "model_perturbed.txt")
# Comprobar que las redes no son iguales
compare_models(individuo_binario, model_perturbed)

Porcentaje de 1s: 30.00%
Porcentaje de 0s: 70.00%
Porcentaje de 1s después de mutación: 99.89%
Porcentaje de 0s después de mutación: 0.11%
Comparando fc1.weight y fc1.weight
¿Los tensores son iguales? True
Comparando fc1.bias y fc1.bias
¿Los tensores son iguales? True
Comparando fc2.weight y fc2.weight
¿Los tensores son iguales? True
Comparando fc2.bias y fc2.bias
¿Los tensores son iguales? True
Comparando fc3.weight y fc3.weight
¿Los tensores son iguales? True
Comparando fc3.bias y fc3.bias
¿Los tensores son iguales? True
Las redes son idénticas.
